In [1]:
import pandas as pd
import numpy as np
import random
from scipy import stats
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Prior Information

In [2]:
# diketahui data kependudukan
df = pd.read_csv("data-jumlah-rukun-tetangga-rukun-warga-per-kelurahan-tahun-2020.csv")
df = df[df["nama_kecamatan"]=="SAWAH BESAR"].reset_index().drop(columns="index")
# informasi tambahan data kependudukan
jumlah_kk = pd.DataFrame([["PASAR BARU",5473],
             ["GUNUNG SAHARI UTARA",7137],
             ["MANGGA DUA SELATAN",12884],
             ["KARANG ANYAR",10971],
             ["KARTINI",9375]], columns=["nama_kelurahan","jumlah_kk"])
# jadikan satu dengan data induk
df = pd.merge(df, jumlah_kk,left_on="nama_kelurahan", right_on="nama_kelurahan", how="left")

In [3]:
df[['nama_kelurahan','jumlah_rw']]

,nama_kelurahan,jumlah_rw
0,PASAR BARU,8
1,KARANG ANYAR,13
2,KARTINI,9
3,GUNUNG SAHARI UTARA,7
4,MANGGA DUA SELATAN,12


### Menentukan Sampel Size

In [4]:
N = len(df["nama_kelurahan"])
n = 4

In [5]:
# menentukan kelurahan (psu) yang dijadikan sampel
ni_list = random.sample(list(df["nama_kelurahan"]),k=n)
df = df[["nama_kelurahan","jumlah_rt","jumlah_rw","jumlah_kk"]]
df = df[df["nama_kelurahan"].isin(ni_list)].reset_index().drop(columns="index")
df

,nama_kelurahan,jumlah_rt,jumlah_rw,jumlah_kk
0,KARANG ANYAR,167,13,10971
1,KARTINI,127,9,9375
2,GUNUNG SAHARI UTARA,99,7,7137
3,MANGGA DUA SELATAN,128,12,12884


In [6]:
Mi_list = list(df["jumlah_rw"])
m = 5

In [7]:
# estimasi jumlah kk per rw
Ki_list_est = []
for i in range(0,len(list(df["jumlah_kk"]))):
    Ki_list_est.append(int(list(df["jumlah_kk"])[i] / Mi_list[i]))
df["estimasi_jumlah_kk_per_rw"] = Ki_list_est
df

,nama_kelurahan,jumlah_rt,jumlah_rw,jumlah_kk,estimasi_jumlah_kk_per_rw
0,KARANG ANYAR,167,13,10971,843
1,KARTINI,127,9,9375,1041
2,GUNUNG SAHARI UTARA,99,7,7137,1019
3,MANGGA DUA SELATAN,128,12,12884,1073


### Melakukan Survei (Dummy)

In [8]:
# generate rw yang terpilih sebagai sampel
rw_terpilih = []
for i in range(0,len(Mi_list)):
    rw_terpilih.append(random.sample(range(1,Mi_list[i]+1),k=m))
rw_terpilih
df["nomor_rw_terpilih"] = rw_terpilih
df

,nama_kelurahan,jumlah_rt,jumlah_rw,jumlah_kk,estimasi_jumlah_kk_per_rw,nomor_rw_terpilih
0,KARANG ANYAR,167,13,10971,843,"[1, 12, 3, 10, 6]"
1,KARTINI,127,9,9375,1041,"[4, 5, 6, 9, 1]"
2,GUNUNG SAHARI UTARA,99,7,7137,1019,"[7, 1, 3, 2, 6]"
3,MANGGA DUA SELATAN,128,12,12884,1073,"[5, 6, 2, 11, 1]"


In [9]:
# ubah menjadi long data
df_new = pd.DataFrame(columns=["nama_kelurahan", "jumlah_rw", "jumlah_kk","estimasi_jumlah_kk_per_rw","nomor_rw_terpilih"])
for j in range(0,len(df)):
    for i in range(0,len(df["nomor_rw_terpilih"][j])):
        x1 = df["nama_kelurahan"]
        x2 = df["jumlah_rw"]
        x3 = df["jumlah_kk"]
        x4 = df["estimasi_jumlah_kk_per_rw"]
        y = df["nomor_rw_terpilih"]
        baris = pd.DataFrame({"nama_kelurahan":x1[j],
                              "jumlah_rw":x2[j],
                              "jumlah_kk":x3[j],
                              "estimasi_jumlah_kk_per_rw":x4[j],
                              "nomor_rw_terpilih":y[j][i]},
                             index=range(0,1))
        df_new = pd.concat([df_new, baris], axis=0, ignore_index=True )
df_new

,nama_kelurahan,jumlah_rw,jumlah_kk,estimasi_jumlah_kk_per_rw,nomor_rw_terpilih
0,KARANG ANYAR,13,10971,843,1
1,KARANG ANYAR,13,10971,843,12
2,KARANG ANYAR,13,10971,843,3
3,KARANG ANYAR,13,10971,843,10
4,KARANG ANYAR,13,10971,843,6
5,KARTINI,9,9375,1041,4
6,KARTINI,9,9375,1041,5
7,KARTINI,9,9375,1041,6
8,KARTINI,9,9375,1041,9
9,KARTINI,9,9375,1041,1


In [10]:
# generate jumlah sampel kk per rw
li_list = []
for i in range(0,len(list(df_new["estimasi_jumlah_kk_per_rw"]))):
    li_list.append(random.randint(30,int(list(df_new["estimasi_jumlah_kk_per_rw"])[i]/15)))
df_new["responden_kk_terekam"] = li_list
df_new

,nama_kelurahan,jumlah_rw,jumlah_kk,estimasi_jumlah_kk_per_rw,nomor_rw_terpilih,responden_kk_terekam
0,KARANG ANYAR,13,10971,843,1,44
1,KARANG ANYAR,13,10971,843,12,51
2,KARANG ANYAR,13,10971,843,3,51
3,KARANG ANYAR,13,10971,843,10,33
4,KARANG ANYAR,13,10971,843,6,35
5,KARTINI,9,9375,1041,4,31
6,KARTINI,9,9375,1041,5,45
7,KARTINI,9,9375,1041,6,40
8,KARTINI,9,9375,1041,9,43
9,KARTINI,9,9375,1041,1,30


In [11]:
# generate survei jumlah konsumsi galon per kk berdasarkan jumlah responden yg ditentukan
konsumsi_galon = []
for i in range(0, len(li_list)):
    y = []
    for j in range(0,li_list[i]):
        y.append(random.randint(14,15))
    konsumsi_galon.append(y)

# gabungkan hasil survei dengan dataframe
nama_kolom = []
i = 1
size_of_list = len(max(konsumsi_galon, key=len))
while i < size_of_list+1:
    nama_kolom.append(f'k{i}')
    i+=1
df_konsumsi_galon = pd.DataFrame(konsumsi_galon,columns=nama_kolom)
raw0 = pd.merge(df_new, df_konsumsi_galon, left_index=True, right_index=True)
raw0

,nama_kelurahan,jumlah_rw,jumlah_kk,estimasi_jumlah_kk_per_rw,nomor_rw_terpilih,responden_kk_terekam,k1,k2,k3,k4,k5,k6,k7,k8,k9,k10,k11,k12,k13,k14,k15,k16,k17,k18,k19,k20,k21,k22,k23,k24,k25,k26,k27,k28,k29,k30,k31,k32,k33,k34,k35,k36,k37,k38,k39,k40,k41,k42,k43,k44,k45,k46,k47,k48,k49,k50,k51,k52,k53,k54,k55,k56,k57,k58,k59,k60,k61,k62,k63,k64,k65,k66,k67
0,KARANG ANYAR,13,10971,843,1,44,14,14,15,15,14,15,15,14,15,14,15,14,15,15,14,14,15,15,14,15,14,14,15,14,15,14,14,14,15,15,15.000,15.000,14.000,14.000,14.000,14.000,14.000,14.000,14.000,15.000,15.000,14.000,14.000,15.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KARANG ANYAR,13,10971,843,12,51,14,15,14,14,14,15,14,14,14,15,14,15,14,14,15,15,14,15,15,14,14,14,15,14,15,15,15,14,14,15,14.000,15.000,14.000,14.000,14.000,14.000,14.000,15.000,14.000,15.000,15.000,14.000,15.000,15.000,14.000,14.000,15.000,14.000,14.000,15.000,15.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KARANG ANYAR,13,10971,843,3,51,14,15,14,15,14,15,15,15,15,14,15,15,15,15,14,15,14,14,15,15,14,15,14,14,15,15,14,15,14,15,15.000,14.000,15.000,14.000,15.000,14.000,14.000,15.000,15.000,15.000,15.000,15.000,15.000,14.000,14.000,14.000,15.000,15.000,15.000,14.000,14.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KARANG ANYAR,13,10971,843,10,33,14,14,14,15,14,15,14,14,15,14,15,15,15,14,14,14,15,15,14,14,15,14,14,14,15,15,15,15,14,14,15.000,15.000,15.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KARANG ANYAR,13,10971,843,6,35,15,15,14,14,15,14,14,15,14,14,15,14,15,14,14,15,14,15,14,14,14,15,15,14,14,14,14,15,15,14,15.000,14.000,15.000,14.000,14.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,KARTINI,9,9375,1041,4,31,15,14,15,15,14,15,14,14,14,14,15,14,15,15,14,14,15,14,15,15,14,15,14,15,14,14,15,15,15,15,15.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,KARTINI,9,9375,1041,5,45,15,14,14,14,14,15,14,15,14,14,14,14,14,14,14,14,14,14,15,14,14,14,15,15,15,15,14,14,14,14,15.000,15.000,14.000,15.000,15.000,15.000,14.000,14.000,15.000,15.000,15.000,15.000,15.000,14.000,14.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,KARTINI,9,9375,1041,6,40,15,14,15,15,14,15,14,14,14,14,15,14,15,14,15,14,15,14,15,14,15,14,14,14,14,15,14,15,15,14,15.000,15.000,14.000,15.000,14.000,14.000,15.000,15.000,15.000,15.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,KARTINI,9,9375,1041,9,43,15,14,15,15,15,15,14,14,15,15,14,14,14,14,14,14,14,15,14,14,14,14,15,15,15,15,14,15,14,14,15.000,14.000,15.000,14.000,14.000,15.000,14.000,15.000,14.000,15.000,14.000,14.000,15.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,KARTINI,9,9375,1041,1,30,14,14,15,15,15,15,14,15,15,15,15,15,15,15,14,15,15,15,15,15,14,15,15,14,15,15,14,14,14,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# ubah menjadi long data
id_vars = raw0.iloc[:,0:6].columns.to_list()
value_vars = raw0.iloc[:,6:].columns.tolist()
raw0 = raw0.melt(id_vars=id_vars, value_vars=value_vars, var_name='sampel_kk', value_name='konsumsi_galon').dropna().reset_index().drop(columns='index')
raw0

,nama_kelurahan,jumlah_rw,jumlah_kk,estimasi_jumlah_kk_per_rw,nomor_rw_terpilih,responden_kk_terekam,sampel_kk,konsumsi_galon
0,KARANG ANYAR,13,10971,843,1,44,k1,14.000
1,KARANG ANYAR,13,10971,843,12,51,k1,14.000
2,KARANG ANYAR,13,10971,843,3,51,k1,14.000
3,KARANG ANYAR,13,10971,843,10,33,k1,14.000
4,KARANG ANYAR,13,10971,843,6,35,k1,15.000
...,...,...,...,...,...,...,...,...
899,GUNUNG SAHARI UTARA,7,7137,1019,6,67,k63,14.000
900,GUNUNG SAHARI UTARA,7,7137,1019,6,67,k64,15.000
901,GUNUNG SAHARI UTARA,7,7137,1019,6,67,k65,14.000
902,GUNUNG SAHARI UTARA,7,7137,1019,6,67,k66,15.000


In [13]:
# hitung jumlah konsumsi hasil survei per rw
raw = raw0.drop(columns=['jumlah_kk','sampel_kk'])
raw = raw.groupby(by=raw.columns[:5].to_list()).sum().reset_index()
raw.rename(columns={'konsumsi_galon':'jumlah_konsumsi_galon_sampel_rw'},inplace=True)
raw

,nama_kelurahan,jumlah_rw,estimasi_jumlah_kk_per_rw,nomor_rw_terpilih,responden_kk_terekam,jumlah_konsumsi_galon_sampel_rw
0,GUNUNG SAHARI UTARA,7,1019,1,49,712.000
1,GUNUNG SAHARI UTARA,7,1019,2,55,792.000
2,GUNUNG SAHARI UTARA,7,1019,3,41,597.000
3,GUNUNG SAHARI UTARA,7,1019,6,67,974.000
4,GUNUNG SAHARI UTARA,7,1019,7,39,562.000
5,KARANG ANYAR,13,843,1,44,636.000
6,KARANG ANYAR,13,843,3,51,744.000
7,KARANG ANYAR,13,843,6,35,504.000
8,KARANG ANYAR,13,843,10,33,478.000
9,KARANG ANYAR,13,843,12,51,736.000


### Hitung Estimator Total Parameter

$$
\widehat{Y}= \frac{N}{n} \sum_{i=1}^{n}\left \{ \frac{M_{i}}{m_{i}}\sum_{j=1}^{m_{i}}\left ( \frac{K_{ij}}{k_{ij}}\sum_{l=1}^{k_{ij}} y_{ijl} \right ) \right \}
$$


Di sini `jumlah_konsumsi_galon_sampel_rw` merupakan $\sum_{l=1}^{k_{ij}} y_{ijl}$
\
Di mana `responden_kk_terekam` adalah $k_{ij}$
\
Dengan `estimasi_jumlah_kk_per_rw` sebagai $K_{ij}$
\
Sehingga terhitung kolom baru `estimasi_galon_per_rw` sebagai $\frac{K_{ij}}{k_{ij}}\sum_{l=1}^{k_{ij}} y_{ijl}$

In [14]:
raw['estimasi_galon_per_rw'] = (raw['estimasi_jumlah_kk_per_rw']/raw['responden_kk_terekam'])*raw['jumlah_konsumsi_galon_sampel_rw']
raw2 = raw
raw.head()

,nama_kelurahan,jumlah_rw,estimasi_jumlah_kk_per_rw,nomor_rw_terpilih,responden_kk_terekam,jumlah_konsumsi_galon_sampel_rw,estimasi_galon_per_rw
0,GUNUNG SAHARI UTARA,7,1019,1,49,712.000,14806.694
1,GUNUNG SAHARI UTARA,7,1019,2,55,792.000,14673.600
2,GUNUNG SAHARI UTARA,7,1019,3,41,597.000,14837.634
3,GUNUNG SAHARI UTARA,7,1019,6,67,974.000,14813.522
4,GUNUNG SAHARI UTARA,7,1019,7,39,562.000,14684.051


$$
\widehat{Y}= \frac{N}{n} \sum_{i=1}^{n}\left \{ \frac{M_{i}}{m_{i}}\sum_{j=1}^{m_{i}}\left ( \frac{K_{ij}}{k_{ij}}\sum_{l=1}^{k_{ij}} y_{ijl} \right ) \right \}
$$


kolom `rerata_galon_per_kelurahan_sampel` merupakan nilai $\frac {\sum_{j=1}^{m_{i}}\left ( \frac{K_{ij}}{k_{ij}}\sum_{l=1}^{k_{ij}} y_{ijl} \right )}{m_{i}}$
\
kolom `jumlah_rw` merupakan $M_{i}$

In [15]:
raw = raw.drop(columns=['estimasi_jumlah_kk_per_rw','responden_kk_terekam','jumlah_konsumsi_galon_sampel_rw'])
raw = (raw.groupby(by=['nama_kelurahan','jumlah_rw'])['estimasi_galon_per_rw'].sum() / raw.groupby(by=['nama_kelurahan','jumlah_rw']).size()).reset_index()
raw.rename( columns={0:'rerata_galon_per_kelurahan_sampel'}, inplace=True )

$$
\widehat{Y}= \frac{N}{n} \sum_{i=1}^{n}\left \{ \frac{M_{i}}{m_{i}}\sum_{j=1}^{m_{i}}\left ( \frac{K_{ij}}{k_{ij}}\sum_{l=1}^{k_{ij}} y_{ijl} \right ) \right \}
$$


`estimasi_galon_per_kelurahan` merupakan nilai $\frac{M_{i}}{m_{i}}\sum_{j=1}^{m_{i}}\left ( \frac{K_{ij}}{k_{ij}}\sum_{l=1}^{k_{ij}} y_{ijl} \right )$

In [16]:
raw['estimasi_galon_per_kelurahan'] = raw['rerata_galon_per_kelurahan_sampel'] * raw['jumlah_rw']
raw = raw.drop(columns=['jumlah_rw','rerata_galon_per_kelurahan_sampel'])
raw

,nama_kelurahan,estimasi_galon_per_kelurahan
0,GUNUNG SAHARI UTARA,103341.702
1,KARANG ANYAR,158596.460
2,KARTINI,136019.606
3,MANGGA DUA SELATAN,186189.568


kemudian kita bisa dapatkan nilai $\widehat{Y}$ dengan `Y_hat`

In [17]:
Y_hat = raw['estimasi_galon_per_kelurahan'].sum() * (N/n)

print(f'estimasi jumlah konsumsi galon di kecamatan sawah besar adalah {int(Y_hat)} galon per bulan')

estimasi jumlah konsumsi galon di kecamatan sawah besar adalah 730184 galon per bulan


### Hitung Varian Total Estimator

$$
\widehat{V}(\widehat{Y}) = N(N-n)\frac{{s_{1}}^{2}}{n}+\frac{N}{n}\sum_{i=1}^{n}M_{i}(M_{i}-m_{i})\frac{{s_{i}}^{2}}{m_{i}}+\frac{N}{n}\sum_{i=1}^{n}\frac{M_{i}}{m_{i}}\sum_{j=1}^{m_{i}}K_{ij}(K_{ij}-k_{ij})\frac{{s_{ij}}^{2}}{k_{ij}}
$$

$$
{s_{1}}^{2}=\frac{\sum_{i=1}^{n}\left ( y_{i}-\frac{\widehat{Y}}{n} \right )^{2}}{n-1}
$$

$$
{s_{i}}^{2}=\frac{\sum_{j=1}^{m_{i}}\left ( y_{ij}-\frac{y_{i}}{m_{i}} \right )^{2}}{m_{i}-1}
$$

$$
{s_{ij}}^{2}=\frac{{{K_{ij}}^{2}}}{{k_{ij}}^{2}}\sum_{l=1}^{k_{ij}}\left ( y_{ijl}-\bar{y}_{ij} \right )^{2}
$$

In [18]:
raw0.head()

,nama_kelurahan,jumlah_rw,jumlah_kk,estimasi_jumlah_kk_per_rw,nomor_rw_terpilih,responden_kk_terekam,sampel_kk,konsumsi_galon
0,KARANG ANYAR,13,10971,843,1,44,k1,14.000
1,KARANG ANYAR,13,10971,843,12,51,k1,14.000
2,KARANG ANYAR,13,10971,843,3,51,k1,14.000
3,KARANG ANYAR,13,10971,843,10,33,k1,14.000
4,KARANG ANYAR,13,10971,843,6,35,k1,15.000


In [19]:
raw0 = raw0.drop(columns=['jumlah_kk','sampel_kk'])

In [20]:
raw0.groupby(by=raw0.columns[:5].to_list()).sum().reset_index().rename(columns={'konsumsi_galon':'jumlah_konsumsi_galon'}).head()

,nama_kelurahan,jumlah_rw,estimasi_jumlah_kk_per_rw,nomor_rw_terpilih,responden_kk_terekam,jumlah_konsumsi_galon
0,GUNUNG SAHARI UTARA,7,1019,1,49,712.000
1,GUNUNG SAHARI UTARA,7,1019,2,55,792.000
2,GUNUNG SAHARI UTARA,7,1019,3,41,597.000
3,GUNUNG SAHARI UTARA,7,1019,6,67,974.000
4,GUNUNG SAHARI UTARA,7,1019,7,39,562.000


In [21]:
raw1 = raw0.groupby(by=raw0.columns[:5].to_list()).mean().reset_index().rename(columns={'konsumsi_galon':'mean_konsumsi_galon'})[['nama_kelurahan','nomor_rw_terpilih','mean_konsumsi_galon']]
x = pd.merge(raw0, raw1, how='right',on=['nama_kelurahan','nomor_rw_terpilih'])
x['varians'] = (x['konsumsi_galon']-x['mean_konsumsi_galon'])**2
x = pd.DataFrame(
    x.groupby(['nama_kelurahan','estimasi_jumlah_kk_per_rw','nomor_rw_terpilih'])['varians'].sum()
     *((x.groupby(['nama_kelurahan','nomor_rw_terpilih'])['estimasi_jumlah_kk_per_rw'].mean())**2
     /(x.groupby(['nama_kelurahan','estimasi_jumlah_kk_per_rw','nomor_rw_terpilih'])['varians'].size())**2)
).reset_index()        
x.rename(columns={0:'varians_total_galon_per_rw'},inplace=True)
x.head()   

,nama_kelurahan,estimasi_jumlah_kk_per_rw,nomor_rw_terpilih,varians_total_galon_per_rw
0,GUNUNG SAHARI UTARA,1019,1,5277.902
1,GUNUNG SAHARI UTARA,1019,2,4531.030
2,GUNUNG SAHARI UTARA,1019,3,6237.307
3,GUNUNG SAHARI UTARA,1019,6,3852.904
4,GUNUNG SAHARI UTARA,1019,7,6441.728


In [22]:
s_i_j_square = x

`varians_total_galon_per_rw` merupakan nilai ${s_{ij}}^{2}$

$$
{s_{i}}^{2}=\frac{\sum_{j=1}^{m_{i}}\left ( y_{ij}-\frac{y_{i}}{m_{i}} \right )^{2}}{m_{i}-1}
$$

kita perlu mencari yij, yakni estimasi jumlah galon per rw

In [23]:
raw2

,nama_kelurahan,jumlah_rw,estimasi_jumlah_kk_per_rw,nomor_rw_terpilih,responden_kk_terekam,jumlah_konsumsi_galon_sampel_rw,estimasi_galon_per_rw
0,GUNUNG SAHARI UTARA,7,1019,1,49,712.000,14806.694
1,GUNUNG SAHARI UTARA,7,1019,2,55,792.000,14673.600
2,GUNUNG SAHARI UTARA,7,1019,3,41,597.000,14837.634
3,GUNUNG SAHARI UTARA,7,1019,6,67,974.000,14813.522
4,GUNUNG SAHARI UTARA,7,1019,7,39,562.000,14684.051
5,KARANG ANYAR,13,843,1,44,636.000,12185.182
6,KARANG ANYAR,13,843,3,51,744.000,12297.882
7,KARANG ANYAR,13,843,6,35,504.000,12139.200
8,KARANG ANYAR,13,843,10,33,478.000,12210.727
9,KARANG ANYAR,13,843,12,51,736.000,12165.647


In [24]:
y = pd.DataFrame(raw2.groupby(['nama_kelurahan'])['estimasi_galon_per_rw'].mean()).reset_index().rename(columns={'estimasi_galon_per_rw':'rerata_estimasi_galon_per_rw'})

In [25]:
y

,nama_kelurahan,rerata_estimasi_galon_per_rw
0,GUNUNG SAHARI UTARA,14763.100
1,KARANG ANYAR,12199.728
2,KARTINI,15113.290
3,MANGGA DUA SELATAN,15515.797


In [26]:
y = pd.merge(raw2,y,how='right',on='nama_kelurahan')

In [27]:
y['varians'] = (y['estimasi_galon_per_rw']-y['rerata_estimasi_galon_per_rw'])**2

In [28]:
y

,nama_kelurahan,jumlah_rw,estimasi_jumlah_kk_per_rw,nomor_rw_terpilih,responden_kk_terekam,jumlah_konsumsi_galon_sampel_rw,estimasi_galon_per_rw,rerata_estimasi_galon_per_rw,varians
0,GUNUNG SAHARI UTARA,7,1019,1,49,712.000,14806.694,14763.100,1900.397
1,GUNUNG SAHARI UTARA,7,1019,2,55,792.000,14673.600,14763.100,8010.311
2,GUNUNG SAHARI UTARA,7,1019,3,41,597.000,14837.634,14763.100,5555.288
3,GUNUNG SAHARI UTARA,7,1019,6,67,974.000,14813.522,14763.100,2542.383
4,GUNUNG SAHARI UTARA,7,1019,7,39,562.000,14684.051,14763.100,6248.753
5,KARANG ANYAR,13,843,1,44,636.000,12185.182,12199.728,211.583
6,KARANG ANYAR,13,843,3,51,744.000,12297.882,12199.728,9634.336
7,KARANG ANYAR,13,843,6,35,504.000,12139.200,12199.728,3663.603
8,KARANG ANYAR,13,843,10,33,478.000,12210.727,12199.728,120.991
9,KARANG ANYAR,13,843,12,51,736.000,12165.647,12199.728,1161.490


In [29]:
s_i_square = pd.DataFrame(y.groupby(by=['nama_kelurahan'])['varians'].sum()/y.groupby(by=['nama_kelurahan'])['varians'].size()).rename(columns={'varians':'varians_total_galon_per_kelurahan'}).reset_index()

In [30]:
s_i_square

,nama_kelurahan,varians_total_galon_per_kelurahan
0,GUNUNG SAHARI UTARA,4851.426
1,KARANG ANYAR,2958.400
2,KARTINI,11723.823
3,MANGGA DUA SELATAN,2428.084


`varians_total_galon_per_kelurahan` merupakan ${s_{i}}^2$

$$
{s_{1}}^{2}=\frac{\sum_{i=1}^{n}\left ( y_{i}-\frac{\widehat{Y}}{n} \right )^{2}}{n-1}
$$

In [31]:
s_u_square = raw['estimasi_galon_per_kelurahan'].var(ddof=1)

In [32]:
s_u_square

1231068446.928517

$$
\widehat{V}(\widehat{Y}) = N(N-n)\frac{{s_{1}}^{2}}{n}+\frac{N}{n}\sum_{i=1}^{n}M_{i}(M_{i}-m_{i})\frac{{s_{i}}^{2}}{m_{i}}+\frac{N}{n}\sum_{i=1}^{n}\frac{M_{i}}{m_{i}}\sum_{j=1}^{m_{i}}K_{ij}(K_{ij}-k_{ij})\frac{{s_{ij}}^{2}}{k_{ij}}
$$

$$N(N-n)\frac{{s_{1}}^{2}}{n}$$

In [33]:
s_u_square

1231068446.928517

In [34]:
first_term = N*(N-n)*(s_u_square)/n

In [35]:
s_i_square.rename(columns={'nama_kelurahan':'i','varians_total_galon_per_kelurahan':'(s_i)^2'},inplace=True)
s_i_square

,i,(s_i)^2
0,GUNUNG SAHARI UTARA,4851.426
1,KARANG ANYAR,2958.400
2,KARTINI,11723.823
3,MANGGA DUA SELATAN,2428.084


$$\frac{N}{n}\sum_{i=1}^{n}M_{i}(M_{i}-m_{i})\frac{{s_{i}}^{2}}{m_{i}}$$

In [36]:
# masukkan jumlah sampel rw masing-masing kelurahan
second_term = s_i_square.join(pd.DataFrame(df_new.groupby(by=['nama_kelurahan'])[[]].size()),on='i').rename(columns={0:'m_i'})
# masukkan jumlah keseluruhan rw masing-masing kelurahan
second_term = pd.merge(second_term,df[['nama_kelurahan','jumlah_rw']], how='left',left_on='i',right_on='nama_kelurahan').drop(columns='nama_kelurahan').rename(columns={'jumlah_rw':'M_i'})

In [37]:
second_term['second_term_sum'] = second_term['M_i'] * (second_term['M_i']-second_term['m_i']) * (second_term['(s_i)^2']/second_term['m_i'])

In [38]:
second_term

,i,(s_i)^2,m_i,M_i,second_term_sum
0,GUNUNG SAHARI UTARA,4851.426,5,7,13583.994
1,KARANG ANYAR,2958.400,5,13,61534.727
2,KARTINI,11723.823,5,9,84411.525
3,MANGGA DUA SELATAN,2428.084,5,12,40791.819


In [39]:
second_term = (N/n)*second_term['second_term_sum'].sum()

$$\frac{N}{n}\sum_{i=1}^{n}\frac{M_{i}}{m_{i}}\sum_{j=1}^{m_{i}}K_{ij}(K_{ij}-k_{ij})\frac{{s_{ij}}^{2}}{k_{ij}}$$

In [40]:
third_term = pd.merge(s_i_j_square, df_new[['nama_kelurahan','nomor_rw_terpilih','responden_kk_terekam']],how='left',on=['nama_kelurahan','nomor_rw_terpilih'])

In [41]:
third_term.head()

,nama_kelurahan,estimasi_jumlah_kk_per_rw,nomor_rw_terpilih,varians_total_galon_per_rw,responden_kk_terekam
0,GUNUNG SAHARI UTARA,1019,1,5277.902,49
1,GUNUNG SAHARI UTARA,1019,2,4531.030,55
2,GUNUNG SAHARI UTARA,1019,3,6237.307,41
3,GUNUNG SAHARI UTARA,1019,6,3852.904,67
4,GUNUNG SAHARI UTARA,1019,7,6441.728,39


In [42]:
third_term.rename(columns={'nama_kelurahan':'i',
                           'nomor_rw_terpilih':'j',
                           'estimasi_jumlah_kk_per_rw':'K_ij',
                           'responden_kk_terekam':'k_ij',
                           'varians_total_galon_per_rw':'(s_ij)^2',
                          },inplace=True
                   )

In [43]:
third_term.head()

,i,K_ij,j,(s_ij)^2,k_ij
0,GUNUNG SAHARI UTARA,1019,1,5277.902,49
1,GUNUNG SAHARI UTARA,1019,2,4531.030,55
2,GUNUNG SAHARI UTARA,1019,3,6237.307,41
3,GUNUNG SAHARI UTARA,1019,6,3852.904,67
4,GUNUNG SAHARI UTARA,1019,7,6441.728,39


In [44]:
third_term['third_term_sum'] = third_term['K_ij'] * (third_term['K_ij']-third_term['k_ij']) * (third_term['(s_ij)^2']/third_term['k_ij'])

In [45]:
third_term.head()

,i,K_ij,j,(s_ij)^2,k_ij,third_term_sum
0,GUNUNG SAHARI UTARA,1019,1,5277.902,49,106466052.100
1,GUNUNG SAHARI UTARA,1019,2,4531.030,55,80925510.670
2,GUNUNG SAHARI UTARA,1019,3,6237.307,41,151609467.039
3,GUNUNG SAHARI UTARA,1019,6,3852.904,67,55785905.165
4,GUNUNG SAHARI UTARA,1019,7,6441.728,39,164944568.178


In [46]:
third_term = pd.DataFrame(third_term.groupby(by=['i'])['third_term_sum'].sum()).reset_index()

In [47]:
# masukkan jumlah sampel rw masing-masing kelurahan
third_term = third_term.join(pd.DataFrame(df_new.groupby(by=['nama_kelurahan'])[[]].size()),on='i').rename(columns={0:'m_i'})
# masukkan jumlah keseluruhan rw masing-masing kelurahan
third_term = pd.merge(third_term,df[['nama_kelurahan','jumlah_rw']], how='left',left_on='i',right_on='nama_kelurahan').drop(columns='nama_kelurahan').rename(columns={'jumlah_rw':'M_i'})

In [48]:
third_term['sum2'] = (third_term['M_i']/third_term['m_i'])*third_term['third_term_sum']

In [49]:
third_term = (N/n)*third_term['sum2'].sum()

In [50]:
V_hat_Y_hat = first_term + second_term + third_term

print(f'varians dari total populasi adalah ({np.sqrt(V_hat_Y_hat)})^2')

varians dari total populasi adalah (90626.74636264893)^2


In [51]:
np.sqrt(V_hat_Y_hat)

90626.74636264893

In [52]:
alpha = 5/100
z_alpha2 = stats.norm.ppf(1-alpha/2)
d = z_alpha2*np.sqrt(V_hat_Y_hat)

### Hasil Akhir

In [53]:
f'Jumlah galon yang terjual selama 1 bulan diestimasikan sebanyak {int(Y_hat)} galon dengan margin of error sebesar {int(d)} galon'

'Jumlah galon yang terjual selama 1 bulan diestimasikan sebanyak 730184 galon dengan margin of error sebesar 177625 galon'

asumsi
market share (berapa saingan di dalam 1 rw)
margin per galon

In [67]:
(((Y_hat)/42)*(20/100))*2000

6954134.950918065

### two-stage method

In [55]:
raw0

,nama_kelurahan,jumlah_rw,estimasi_jumlah_kk_per_rw,nomor_rw_terpilih,responden_kk_terekam,konsumsi_galon
0,KARANG ANYAR,13,843,1,44,14.000
1,KARANG ANYAR,13,843,12,51,14.000
2,KARANG ANYAR,13,843,3,51,14.000
3,KARANG ANYAR,13,843,10,33,14.000
4,KARANG ANYAR,13,843,6,35,15.000
...,...,...,...,...,...,...
899,GUNUNG SAHARI UTARA,7,1019,6,67,14.000
900,GUNUNG SAHARI UTARA,7,1019,6,67,15.000
901,GUNUNG SAHARI UTARA,7,1019,6,67,14.000
902,GUNUNG SAHARI UTARA,7,1019,6,67,15.000


In [56]:
alt_data = raw0.groupby(['nama_kelurahan','nomor_rw_terpilih','responden_kk_terekam'])['konsumsi_galon'].sum().reset_index()
alt_data = pd.merge(df[['nama_kelurahan','jumlah_kk']],alt_data,on='nama_kelurahan')

In [57]:
alt_data.head()

,nama_kelurahan,jumlah_kk,nomor_rw_terpilih,responden_kk_terekam,konsumsi_galon
0,KARANG ANYAR,10971,1,44,636.000
1,KARANG ANYAR,10971,3,51,744.000
2,KARANG ANYAR,10971,6,35,504.000
3,KARANG ANYAR,10971,10,33,478.000
4,KARANG ANYAR,10971,12,51,736.000


In [58]:
N = 5
n = 4

M_list = alt_data.groupby('nama_kelurahan')['jumlah_kk'].mean().to_list()
m_list = alt_data.groupby('nama_kelurahan')['responden_kk_terekam'].sum().to_list()

y_bar_list = alt_data.groupby('nama_kelurahan')['konsumsi_galon'].mean().to_list()
s_i_square_list = alt_data.groupby('nama_kelurahan')['konsumsi_galon'].var().to_list()

In [59]:
# calculate the mu_hat_r
numerator = 0
denumerator = 0
for i in range(n):
    numerator += M_list[i] * y_bar_list[i]
    denumerator += M_list[i]
mu_hat_r = numerator/denumerator

print(f"Estimate of average online transaction per rw: {mu_hat_r:.0f}")

Estimate of average online transaction per rw: 655


In [60]:
# Calculate the first term
first_term = 0
for i in range(n):
    first_term += (M_list[i]*y_bar_list[i] - M_list[i]*mu_hat_r)**2
first_term *= (N*(N-n))/(n*(n-1))


# Calculate the second term
second_term = 0
for i in range(n):
    second_term += M_list[i]*(M_list[i]-m_list[i])*(s_i_square_list[i]/m_list[i])
second_term *= (N/n)


# Calculate var_tau_hat_r
var_tau_hat_r = first_term + second_term
print(f"Variance of total estimator by ratio: ({np.sqrt(var_tau_hat_r):.0f} transactions)^2")


# Finally calculate var_mu_hat_r
var_mu_hat_r = (1/((np.mean(M_list)*N)**2)) * var_tau_hat_r
print(f"Variance of mean estimator by ratio: ({np.sqrt(var_mu_hat_r):.0f} transactions)^2")

Variance of total estimator by ratio: (975055 transactions)^2
Variance of mean estimator by ratio: (19 transactions)^2


In [61]:
# calculate z_stat @ 95% CI
alpha = 0.05
z_stat = stats.norm.ppf(1 - alpha/2.)
print(f"z_stat                                  : {z_stat:.2f}")

# calculate the margin of error (d)
d = z_stat * np.sqrt(var_mu_hat_r)
print(f"margin of error of mean estimator (d)  : {d:.0f} transactions")

z_stat                                  : 1.96
margin of error of mean estimator (d)  : 38 transactions


In [62]:
# Calculate the estimate total number of supermarkets (M_hat)
M_hat = (N/n) * np.sum(M_list)
print(f"Estimate total number of supermarkets: {M_hat:.0f}")

Estimate total number of supermarkets: 50459


In [63]:
# Calculate tau_hat_r
tau_hat_r = M_hat * mu_hat_r
print(f"Estimate total of online transaction in all supermarkets: {tau_hat_r:.0f}")

Estimate total of online transaction in all supermarkets: 33048048


In [64]:
# Calculate the first term
first_term = 0
for i in range(n):
    first_term += (M_list[i]*y_bar_list[i] - M_list[i]*mu_hat_r)**2
first_term *= (N*(N-n))/(n*(n-1))


# Calculate the second term
second_term = 0
for i in range(n):
    second_term += M_list[i]*(M_list[i]-m_list[i])*(s_i_square_list[i]/m_list[i])
second_term *= (N/n)


# Calculate var_tau_hat_r
var_tau_hat_r = first_term + second_term
print(f"Variance of total estimator by ratio: ({np.sqrt(var_tau_hat_r):.0f} transactions)^2")

Variance of total estimator by ratio: (975055 transactions)^2


In [65]:
# calculate z_stat @ 95% CI
alpha = 0.05
z_stat = stats.norm.ppf(1 - alpha/2.)
print(f"z_stat                                  : {z_stat:.2f}")

# calculate the margin of error (d)
d = z_stat * np.sqrt(var_tau_hat_r)
print(f"margin of error of total estimator (d)  : {d:.0f} transactions")

z_stat                                  : 1.96
margin of error of total estimator (d)  : 1911073 transactions


In [66]:
# Calculate proportion of online transaction
trans_tot = 1500000
p_online = mu_hat_r / trans_tot

print(f"proportion of online transaction: {p_online*100:.2f} %")

proportion of online transaction: 0.04 %
